In [ ]:
import os, warnings
warnings.filterwarnings('ignore')
from tensorflow.keras.utils import image_dataset_from_directory

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import layers
import keras
from keras.layers import Dense, Flatten, Normalization, Dropout, Conv2D, MaxPooling2D, RandomFlip, RandomRotation, RandomZoom, BatchNormalization, Activation, AveragePooling2D
from keras.models import Sequential
from keras.losses import SparseCategoricalCrossentropy, CategoricalCrossentropy
from keras.callbacks import EarlyStopping
from keras.utils import np_utils
from keras import utils

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import math

from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Input
import requests

In [ ]:
model_url = 'http://'
file_path = 'custom_weights.h5'

response = requests.get(model_url)

with open(file_path, 'wb') as file:
    file.write(response.content)

In [ ]:
num_epochs = 10
input_shape = (224, 224, 3)

custom_model = Sequential([
                    Conv2D(128, (3, 3), input_shape=input_shape, padding="same", kernel_regularizer="l2"),
                    BatchNormalization(),
                    Activation("relu"),
                    MaxPooling2D((2, 2)),
                    Conv2D(128, (3, 3), padding="same", kernel_regularizer="l2"),
                    Activation("relu"),
                    BatchNormalization(),
                    MaxPooling2D((2, 2)),
                    Dropout(.4),
                    Conv2D(128, (3, 3), padding="same", kernel_regularizer="l2"),
                    Activation("relu"),
                    BatchNormalization(),
                    MaxPooling2D((2, 2)),
                    Dropout(.4),
                    Flatten(),
                    Dense(128, activation='relu', kernel_regularizer="l2"),
                    Dense(15, activation="softmax")])

# use this variable to train the model or to load the model in from weights
custom_trainable = False

# Set the 'trainable' attribute to False
for layer in custom_model.layers:
    layer.trainable = custom_trainable

custom_model.build(input_shape=(None, input_shape))
custom_model.summary()

In [ ]:
custom_model.load_weights('custom_weights.h5')